# Dowker persistence example
* This notebook shows an example computation of Dowker persistence.

In [1]:
include("dowker_persistence.jl")
using .Dowker
using Eirene
using Plots
using CSV
using Distances
using DelimitedFiles
using JLD
using PersistenceDiagrams

WebIO._IJuliaInit()

┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA /Users/irisyoon/.julia/packages/ORCA/U5XaN/src/ORCA.jl:8


# 1. Compute Dowker persistence

### Load example points

In [2]:
# load points
P1 = readdlm("toy_example/R1_coordinates.csv", ',', Float64)
P2 = readdlm("toy_example/R2_coordinates.csv", ',', Float64);

In [3]:
# plot
plot_P_Q(P1, P2; 
        P_label = "P1", 
        Q_label = "P2",
        aspect_ratio = :equal, )

<!DOCTYPE html>
 
 
 Plots.jl

### Compute distance matrices

In [4]:
D1, D2, D_P1_P2, D_P2_P1 = compute_distance(P1, P2);

### Compute (usual) Vietoris-Rips persistence

In [5]:
C1 = eirene(D1)
C1_barcode = Eirene.barcode(C1, dim = 1)
C2 = eirene(D2)
C2_barcode = Eirene.barcode(C2, dim = 1);

In [6]:
p1 = plot_PD(C1_barcode, title = "P1")
p2 = plot_PD(C2_barcode, title = "P2")
plot(p1, p2, layout = grid(1,2))

<!DOCTYPE html>
 
 
 Plots.jl

### Compute Dowker persistence

In [7]:
W_P1_P2 = compute_Witness_persistence(D_P1_P2, maxdim = 1)
W_barcode = Eirene.barcode(W_P1_P2["eirene_output"], dim = 1);

In [8]:
# plot Dowker persistence diagram
p = plot_PD(W_barcode, title = "Dowker persistence diagram")
plot(p)

<!DOCTYPE html>
 
 
 Plots.jl

Plot a cycle representative of the Dowker persistence diagram

In [9]:
# get cycle representative
cr = get_Witness_cyclerep(W_P1_P2, class_num = 1);

In [10]:
plot_cycle(P1, P2, cycle = cr; aspect_ratio = :equal, P_label = "P1", Q_label = "P2")

<!DOCTYPE html>
 
 
 Plots.jl

### Compute persistence image

In [11]:
# convert format of barcodes from array to persistencediagram 
n = size(W_barcode,1)
W_diag = PersistenceDiagram([(W_barcode[i,1], W_barcode[i,2]) for i = 1:n])

C1_diag = PersistenceDiagram([(C1_barcode[i,1], C1_barcode[i,2]) for i = 1:size(C1_barcode,1)])
C2_diag = PersistenceDiagram([(C2_barcode[i,1], C2_barcode[i,2]) for i = 1:size(C2_barcode,2)]);


In [12]:
PI = PersistenceDiagrams.PersistenceImage([W_diag, C1_diag, C2_diag],
    sigma = 0.05,
    size = 100)

100×100 PersistenceImage(
  distribution = PersistenceDiagrams.Binormal(0.05),
  weight = PersistenceDiagrams.DefaultWeightingFunction(0.47198741667412825),
)

In [13]:
p1 = heatmap(PI(C1_diag), title = "VR(P1)")
p2 = heatmap(PI(C2_diag), title = "VR(P2)")
p3 = heatmap(PI(W_diag), title = "W(P1, P2)")
plot(p1, p2, p3, layout = grid(1,3), size = (800, 300))

<!DOCTYPE html>
 
 
 Plots.jl

# 2. Dowker duality
* Note that the input to Witness persistence above was D_P1_P2, where the rows correspond to P1 and the columns correspond to P2.
* What happens if we compute the Witness persistence using the transpose of D_P1_P2? 
    * Let D_P2_P1 be the transpose
    * Rows: P2
    * Columns: P1
    * The result should be the same

Compute Dowker persistence

In [14]:
W_P2_P1 = compute_Witness_persistence(D_P2_P1, maxdim = 1)
W_barcode = Eirene.barcode(W_P2_P1["eirene_output"], dim = 1);

In [15]:
# plot Dowker persistence diagram
p = plot_PD(W_barcode, title = "")
plot(p)

<!DOCTYPE html>
 
 
 Plots.jl

In [16]:
# get cycle representative
cr_P2_P1 = get_Witness_cyclerep(W_P2_P1, class_num = 1);

In [17]:
plot_cycle(P1, P2, cycle = cr_P2_P1; aspect_ratio = :equal, cycle_loc = "Q", P_label = "P1", Q_label = "P2")

<!DOCTYPE html>
 
 
 Plots.jl

* Using the transpose matrix `D_P2_P1` results in the same Dowker barcodes.
* Based on the cycle representatives, one can see that the Dowker barcode picks out common / shared features between P1 and P2